In [1]:
struct Calibration
    σ::Float64           # elasticity of relative demand with respect to price         
    γ::Vector{Float64}   # intensity in each good
    ϵ::Vector{Float64}   # elasticity of relative demand with respect to income in luxury good sector
    κ::Vector{Float64}   # For Stone and Geary
    g::Vector{Float64}
end

cal = Calibration(
    0.5,                    # sigma=0.5 completementary goods
    [1/3, 1/3, 1/3],        #intensity
    [0.6, 1.0, 1.65],       #ELASTICITIES
    [0.05, 0.03, 0.02],     #For Stone and Geary
    [1.05, 1.02, 1.01]      # price dynamics
)

ρ=(cal.σ-1)/cal.σ

-1.0

In [51]:
p=[5.0, 10.0, 15.0]
Co=[5.0, 1.0, 10.0]


3-element Vector{Float64}:
  5.0
  1.0
 10.0

In [166]:
using JuMP, Ipopt
m=Model(Ipopt.Optimizer)
@variable(m, C[1:3])
@variable(m, U)
E=sum(p[i]*C[i] for i=1:length(C))
Exp=sum(p[i]*Co[i] for i=1:length(Co))

185.0

In [167]:
@NLobjective(m, Max, U)
@NLconstraints m begin
    Utility, 1==sum(cal.γ[i]^(1/cal.σ)*(C[i]/U^((1-cal.σ)*cal.σ*cal.ϵ[i]))^ρ for i=1:length(C))
    Expenditure, Exp==sum(p[i]*C[i] for i = 1:length(C))   
    P1, 0<=C[1]<=10
    P2, 0<=C[2]
    P3, 0<=C[3] 
end

(1.0 - (0.3333333333333333 ^ (1.0 / 0.5) * (C[1] / U ^ ((1.0 - 0.5) * 0.5 * 0.6)) ^ -1.0 + 0.3333333333333333 ^ (1.0 / 0.5) * (C[2] / U ^ ((1.0 - 0.5) * 0.5 * 1.0)) ^ -1.0 + 0.3333333333333333 ^ (1.0 / 0.5) * (C[3] / U ^ ((1.0 - 0.5) * 0.5 * 1.65)) ^ -1.0) = 0, 185.0 - (5.0 * C[1] + 10.0 * C[2] + 15.0 * C[3]) = 0, 0 ≤ C[1] ≤ 10, 0.0 - C[2] ≤ 0, 0.0 - C[3] ≤ 0)

In [168]:
print(m)

Max U
Subject to
 1.0 - (0.3333333333333333 ^ (1.0 / 0.5) * (C[1] / U ^ ((1.0 - 0.5) * 0.5 * 0.6)) ^ -1.0 + 0.3333333333333333 ^ (1.0 / 0.5) * (C[2] / U ^ ((1.0 - 0.5) * 0.5 * 1.0)) ^ -1.0 + 0.3333333333333333 ^ (1.0 / 0.5) * (C[3] / U ^ ((1.0 - 0.5) * 0.5 * 1.65)) ^ -1.0) = 0
 185.0 - (5.0 * C[1] + 10.0 * C[2] + 15.0 * C[3]) = 0
 0 ≤ C[1] ≤ 10
 0.0 - C[2] ≤ 0
 0.0 - C[3] ≤ 0


In [169]:
optimize!(m)

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        7
Number of nonzeros in inequality constraint Jacobian.:        3
Number of nonzeros in Lagrangian Hessian.............:        7

The Jacobian for the equality constraints contains an invalid number

Number of Iterations....: 0

Number of objective function evaluations             = 0
Number of objective gradient evaluations             = 0
Number of equality constraint evaluations            = 0
Number of inequality constraint evaluations          = 1
Number of equality constraint Jacobian evaluations   = 1
Number of inequality constraint Jacobian evaluations = 0
Number of Lagrangian Hessian evaluations             = 0
Total seconds in IPOPT                               = 0.000

EXIT: Invalid number in NLP function or derivative detected.


In [156]:
value.(C)

3-element Vector{Float64}:
 10.000000099681763
  6.749999975079559
  4.499999983386374

In [28]:
using JuMP, Ipopt
m = Model(Ipopt.Optimizer)

@variable(m, x[1:2])
@NLobjective(m, Min, (x[1]-3)^3 + (x[2]-4)^2)
@NLconstraint(m, (x[1]-1)^2 + (x[2]+1)^3 + exp(-x[1]) <= 1)

JuMP.optimize!(m)

println("** Optimal objective function value = ", JuMP.objective_value(m))
println("** Optimal solution = ", JuMP.value.(x))

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        4

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        1

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -1.1000000e+01 2.00e+00 1.03e+01  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

[0.49239864764413793, -0.4918893479728137]


In [29]:
print(m)

Min (x[1] - 3.0) ^ 3.0 + (x[2] - 4.0) ^ 2.0
Subject to
 ((x[1] - 1.0) ^ 2.0 + (x[2] + 1.0) ^ 3.0 + exp(-x[1])) - 1.0 ≤ 0
